<a href="https://colab.research.google.com/github/AlexKressner/Business_Intelligence/blob/main/ML_Decision_Tree_Random_Forest_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression Tree, Beispiel Immobilienpreise

In [ ]:
import pandas as pd

In [ ]:
data = [
    [60000, 80000, 10000],
    [30000, 70000, 120000],
    [25000, 50000, 40000],
    [85000, 120000, 20000],
    [30000, 65000, 80000]
    ]

In [ ]:
df = pd.DataFrame(data, columns=["Verkaufspreis", "Neupreis", "Kilometerstand"])

In [ ]:
df.head()

In [ ]:
X = df.drop(columns="Verkaufspreis")
y = df["Verkaufspreis"]

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
decision_tree = DecisionTreeRegressor()

In [ ]:
decision_tree.fit(X, y)

In [ ]:
from sklearn.tree import export_graphviz
from IPython.display import Image

In [ ]:
# Hilfsfunktion zur Darstellung des Entscheidungsbaums
def plot_decision_tree(decision_tree:DecisionTreeRegressor, name:str="tree"):
  # Exportieren des Entscheidungsbaums in eine .dot Datei
  export_graphviz(
      decision_tree,
      out_file=f"./{name}.dot",
      feature_names=decision_tree.feature_names_in_,
      rounded=True,
      filled=True
  )

  #Umwandeln der .dot in eine .png Datei
  ! dot -Tpng ./tree.dot -o ./tree.png

  # Visualisierung
  return Image(filename=f"./{name}.png")

In [ ]:
plot_decision_tree(decision_tree)

# Regression Tree, Beispiel Immobilienpreise



## 1 Daten laden

In [ ]:
! git clone https://github.com/AlexKressner/Business_Intelligence

In [ ]:
path = "Business_Intelligence/Daten/ML_Regression/"

In [ ]:
# Datentypen für das Laden der Daten definieren
# Beispiel: Zwar handelt es sich beim Baujahr um eine Zahl, allerdings interessiert uns
# hier nur, ob eine Immobilie in einem Jahr gebaut wurde oder eben nicht.
dtypes = {
    "Verkaufsmonat": "str",
    "Baujahr": "str",
}

In [ ]:
# Relevante Spalten definiere
cols = [
    "Verkaufspreis","Verkaufsmonat","Wohnraum_qm",
    "Keller_qm", "Grundstueck_qm","Baujahr"
    ]

In [ ]:
# Daten laden
data = pd.read_csv(f"{path}house_price.txt", dtype=dtypes, usecols=cols)

## 2 Erster Überblick zum Datensatz

In [ ]:
# Kopfzeile der Daten
data.head()

In [ ]:
# Größe des Datensatzes
data.shape

## 3 Prognose mit einem Entscheidungsbaum

### 3.1 Daten aufbereiten

In [ ]:
# Features, d.h. womit lässt sich der Immobilienpreis vorhersagen!
X = data.drop(columns="Verkaufspreis")

In [ ]:
# Target, d.h. was soll prognostiziert werden!
y = data["Verkaufspreis"]

In [ ]:
# Kategoriale Features umcodieren
X = pd.get_dummies(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Training Algorithmus: X_train (Features), y_train (Target)
# Test des Algorithmus (Bewertung): X_test, y_test
#     1) Algorithmus bekommt Daten (X_test), die er noch nicht kennt & macht Prognose
#     2) Vergleich Prognose mit tatsächlichen Werte (y_test) --> Bewertung

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

### 3.2 Entscheidungsbaum zur Regression
#### 3.2.1 Default Parameter

In [ ]:
decision_tree = DecisionTreeRegressor()

In [ ]:
decision_tree.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
# Fehler auf den Trainingsdaten
mean_absolute_percentage_error(y_train, decision_tree.predict(X_train)) * 100

In [ ]:
# Fehler auf den Testdaten
mean_absolute_percentage_error(y_test, decision_tree.predict(X_test)) * 100

In [ ]:
plot_decision_tree(decision_tree)

#### 3.2.2 Minimale Anzahl von Beobachtungen in Blattknoten
Die Mindestanzahl von Datensätzen, die für ein Blattknoten erforderlich sind. Ein Blattknoten ist ein Endknoten um Entscheidungsbaum, dem keine weiteren Verzweigungen folgen. Ein Teilungspunkt auf beliebiger Tiefe wird nur dann in Betracht gezogen, wenn er mindestens `min_samples_leaf` Beobachtungen in jedem der linken und rechten Zweige hinterlässt.

In [ ]:
decision_tree = DecisionTreeRegressor(min_samples_leaf=10)

In [ ]:
decision_tree.fit(X_train, y_train)

In [ ]:
# Fehler auf den Trainingsdaten
mean_absolute_percentage_error(y_train, decision_tree.predict(X_train)) * 100

In [ ]:
# Fehler auf den Testdaten
mean_absolute_percentage_error(y_test, decision_tree.predict(X_test)) * 100

In [ ]:
plot_decision_tree(decision_tree)

#### 3.2.3 Minimale Anzahl von Beobachtungen für einen Split

In [ ]:
decision_tree = DecisionTreeRegressor(min_samples_split=40)

In [ ]:
decision_tree.fit(X_train, y_train)

In [ ]:
# Fehler auf den Trainingsdaten
mean_absolute_percentage_error(y_train, decision_tree.predict(X_train)) * 100

In [ ]:
# Fehler auf den Testdaten
mean_absolute_percentage_error(y_test, decision_tree.predict(X_test)) * 100

In [ ]:
plot_decision_tree(decision_tree)

#### 3.2.4 Maximale Tiefe des Entscheidungsbaums

In [ ]:
print(f"Aktuelle Tiefe des Entscheidungsbaums: {decision_tree.get_depth()}")

In [ ]:
decision_tree = DecisionTreeRegressor(max_depth=7, min_samples_leaf=10)

In [ ]:
decision_tree.fit(X_train, y_train)

In [ ]:
# Fehler auf den Trainingsdaten
mean_absolute_percentage_error(y_train, decision_tree.predict(X_train)) * 100

In [ ]:
# Fehler auf den Testdaten
mean_absolute_percentage_error(y_test, decision_tree.predict(X_test)) * 100

In [ ]:
plot_decision_tree(decision_tree)

## 4 Bestimmung optimaler Parameter für den Entscheidungsbaum

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Grid definieren
param_grid = {
    "max_depth": [3, 4, 5, 6, 7, 8, 9],
    "min_samples_leaf": [5, 10, 15, 20, 25, 30],
    "min_samples_split": [10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
}

In [ ]:
decision_tree = DecisionTreeRegressor()

In [ ]:
grid = GridSearchCV(
    decision_tree, # verwendetes Modell
    param_grid, # zu untersuchende Parameter
    cv=5, # Anzahl der Folds
    n_jobs=-1, # Parallelisierung der Ausführung
    scoring="neg_mean_squared_error", # Fehlermaß
)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid_search_results = pd.DataFrame(grid.cv_results_)[
    ["mean_test_score", "std_test_score", "params"]
]
grid_search_results.head()

In [ ]:
grid.best_estimator_

In [ ]:
# Auswahl des besten Modells
best_model = grid.best_estimator_

In [ ]:
best_model.get_params()

In [ ]:
# Fehler auf den Trainingsdaten
mean_absolute_percentage_error(y_train, best_model.predict(X_train)) * 100

In [ ]:
# Fehler auf den Testdaten
mean_absolute_percentage_error(y_test, best_model.predict(X_test)) * 100

In [ ]:
plot_decision_tree(best_model)

# Random Forest Regressor, Beispiel Immobilienpreise

In [ ]:
from sklearn.ensemble import RandomForestRegressor

## Random Forest Regressor mit Default-Parametern

In [ ]:
forest = RandomForestRegressor()

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
# Fehler auf den Trainingsdaten
mean_absolute_percentage_error(y_train, forest.predict(X_train)) * 100

In [ ]:
# Fehler auf den Testdaten
mean_absolute_percentage_error(y_test, forest.predict(X_test)) * 100

## Random Forest Regressor mit optimierten Parametern

In [ ]:
# Grid definieren
param_grid = {
    "max_depth": [4, 6, 8, 10, 12, 14, 16],
    "n_estimators": [50, 100, 150, 200]
}

In [ ]:
grid = GridSearchCV(
    forest, # verwendetes Modell
    param_grid, # zu untersuchende Parameter
    cv=5, # Anzahl der Folds
    n_jobs=-1, # Parallelisierung der Ausführung
    scoring="neg_mean_squared_error", # Fehlermaß
)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
best_model = grid.best_estimator_
best_model.get_params()

In [ ]:
# Fehler auf den Trainingsdaten
mean_absolute_percentage_error(y_train, best_model.predict(X_train)) * 100

In [ ]:
# Fehler auf den Testdaten
mean_absolute_percentage_error(y_test, best_model.predict(X_test)) * 100

# Hinweis: Cross-Validation für Zeitreihen

## Klassische Cross-Validation

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" width=700 height=400 >

## Cross-Validation unter Beachtung der Zeitreiheneigenschaft

<img src="https://quantile.app/images/blog_posts/cross_validation/holdout_cv.webp" width=700 height=400 >